### Genetisch algioritme voor optimale hyper-parameters verkrijgen uit de iris dataset

Deze code-cel begint met het importeren van de benodigde Python-bibliotheken en instellingen voor het uitvoeren van hyperparameteroptimalisatie voor een neuraal netwerkmodel. Hier zijn de belangrijkste elementen die in deze cel gebeuren:

1. **Importeren van Numpy en scikit-learn Libraries**:
   - De code importeert `numpy` als `np` voor numerieke berekeningen.
   - Het laadt de Iris-dataset met behulp van de `load_iris` functie van scikit-learn. Deze dataset zal worden gebruikt voor het evalueren van het neuraal netwerkmodel.
   - Het importeert `train_test_split` om de dataset te splitsen in trainings- en testgegevens.
   - Het gebruikt `MLPRegressor` van scikit-learn om een meerlaagse perceptron-neuraal netwerk voor regressie te definiëren en te configureren.

2. **Importeren van de Tabulate-bibliotheek**:
   - De code importeert de `tabulate`-bibliotheek. Deze bibliotheek wordt later gebruikt om de resulterende error metrics op een nette manier als een tabel weer te geven.

3. **Importeren van Regressiemetrieken uit sklearn**
   - De code importeert de `mean_absolute_error`, `mean_squared_error` en `r2_score` scoringsmetrieken om het genetisch regressie algoritme te scoren.

4. **Uitschakelen van Waarschuwingen**:
   - Er wordt een waarschuwingssuppressie geactiveerd om te voorkomen dat waarschuwingen de uitvoer onnodig vervuilen.

Deze voorbereidende stappen zijn cruciaal voor het correct uitvoeren van de hyperparameteroptimalisatie voor het neurale netwerkmodel en voor het weergeven van de resultaten in een overzichtelijke tabel met error metrics aan het einde van het script.


In [1]:
# General imports for the hyperparameter optimization
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

# Import the tabulate library for a nice table at the end of the file for the error metrics
from tabulate import tabulate

# Importeer relevante regressiemetrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Ignoring all warnings because it clutters the space where the results are supposed to go in to
import warnings
warnings.filterwarnings("ignore")


### Gebruik van de Iris Dataset en Dataverdeling

In deze code-cel wordt de Iris-dataset gebruikt en wordt de dataset verdeeld in trainings- en testgegevens. Hier zijn de belangrijkste elementen die in deze cel gebeuren:

1. **Laden van de Iris Dataset**:
   - De code laadt de Iris-dataset met behulp van de `load_iris` functie. De Iris-dataset bevat informatie over irisbloemen en wordt vaak gebruikt voor machinaal leren toepassingen.

2. **Dataverdeling**:
   - De code splitst de gegevens in onafhankelijke variabelen (`X`) en de doelvariabele (`y`). `X` bevat de kenmerken van de bloemen, terwijl `y` de bijbehorende klasselabels bevat.
   - De dataset wordt verder opgesplitst in trainings- en testgegevens met behulp van `train_test_split`. Hierbij wordt 80% van de gegevens toegewezen aan de trainingsset en 20% aan de testset.
   - De `random_state` wordt ingesteld op 42 (“the Ultimate Question of Life, the Universe, and Everything.") om ervoor te zorgen dat de verdeling van de gegevens reproduceerbaar is.

Deze stappen zijn essentieel voor het voorbereiden van de gegevens voor de training en evaluatie van het neurale netwerkmodel. De Iris-dataset wordt gebruikt als de basis voor dit machine learning-assignment.


In [2]:
# We're still using the iris dataset for this assignment
data = load_iris()

# Splitting the data into training and testing data
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Definitie van te Testen Hyperparameters

In deze code-cel worden de hyperparameters gedefinieerd die zullen worden getest bij het optimaliseren van het neurale netwerkmodel. Hier zijn de belangrijkste elementen die in deze cel gebeuren:

1. **Activatiefuncties**:
   - De code definieert een lijst `activation_functions` met verschillende activatiefuncties die zullen worden getest. Deze functies beïnvloeden de overgang van informatie tussen neuronen in het netwerk. De beschikbare activatiefuncties zijn 'logistic', 'tanh', 'relu', en 'identity'.

2. **Grootte van Verborgen Lagen**:
   - De grootte van de verborgen lagen wordt gedefinieerd als een lijst van getallen in `hidden_layer_sizes`. Elke waarde in de lijst vertegenwoordigt de grootte van een verborgen laag in het neurale netwerk. Dit geeft flexibiliteit bij het testen van verschillende architecturen.

3. **Optimalisatie Solvers**:
   - De code definieert een lijst `solvers` met verschillende optimalisatiesolvers die worden gebruikt om de optimale gewichten voor het neurale netwerk te vinden. De beschikbare solvers zijn 'lbfgs', 'sgd', en 'adam'. Het standaard leerpercentage wordt gebruikt voor elk van deze solvers.

Deze definitie van hyperparameters is cruciaal voor het begeleiden van de hyperparameteroptimalisatie van het neurale netwerkmodel. Door verschillende combinaties van activatiefuncties, verborgen laaggroottes en solvers te testen, kan de optimale configuratie van het model worden bepaald.


In [3]:
# Defining the hyperparameters that we want to test
activation_functions = ['logistic', 'tanh', 'relu', 'identity']
# The hidden layer sizes are defined as a list of tuples, where each tuple represents a hidden layer
hidden_layer_sizes = [10, 20, 30, 40]
# The solvers are the optimizers that are used to find the optimal weights
# We're using the default learning rate for each solver
solvers = ['lbfgs', 'sgd', 'adam']

### Evaluatie van Individuen en Genetisch Algoritme

Deze code-cel bevat de implementatie van een functie voor het evalueren van de nauwkeurigheid van individuele neurale netwerkconfiguraties, evenals een genetisch algoritme voor het zoeken naar de beste individuele configuratie. Hier zijn de belangrijkste elementen die in deze cel gebeuren:

**Functie voor Evaluatie van Individuen**:
- De functie `evaluate_individual` ontvangt drie parameters: `activation_function`, `hidden_layer_size`, en `solver`.
- Binnen deze functie wordt een `MLPRegressor`-model geïnitialiseerd en getraind met de opgegeven hyperparameters. Het model maakt gebruik van een enkele verborgen laag met de specificaties.
- De nauwkeurigheid van het model wordt berekend op basis van de testgegevens (`X_test` en `y_test`) en geretourneerd als het evaluatieresultaat.

**Genetisch Algoritme voor Hyperparameteroptimalisatie**:
- De functie `genetic_algorithm` wordt gebruikt om het genetisch algoritme uit te voeren voor hyperparameteroptimalisatie.
- Een initiële populatie van individuen wordt gecreëerd, waarbij de hyperparameters willekeurig worden gekozen uit de gedefinieerde sets van activatiefuncties, verborgen laaggroottes en solvers.
- Het genetisch algoritme wordt vervolgens uitgevoerd over meerdere generaties, waarbij individuen worden geëvalueerd en de beste individuen worden geselecteerd op basis van hun nauwkeurigheid.
- Nieuwe individuen worden gegenereerd door combinaties van ouders en een willekeurige mutatie wordt geïntroduceerd om diversiteit te behouden.
- Het genetisch algoritme zoekt naar het beste individu met de hoogste nauwkeurigheid en retourneert de hyperparameters van dat individu als resultaat.

Deze implementaties zijn cruciaal voor het automatisch zoeken naar de optimale hyperparameters van het neurale netwerkmodel. Het genetisch algoritme evolueert de populatie van individuen om de configuratie te vinden die leidt tot de beste nauwkeurigheid bij regressie.


In [4]:
# The function that evaluates the accuracy of an individual
def evaluate_individual(activation_function, hidden_layer_size, solver):
    clf = MLPRegressor(hidden_layer_sizes=(hidden_layer_size,), activation=activation_function, solver=solver, max_iter=2000, random_state=42)
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    


    return accuracy

# The genetic algorithm that finds the best individual
def genetic_algorithm(population_size, generations):
    population = []

    # Creating the initial population
    for _ in range(population_size):
        activation_function = np.random.choice(activation_functions)
        hidden_layer_size = np.random.choice(hidden_layer_sizes)
        solver = np.random.choice(solvers)
        individual = (activation_function, hidden_layer_size, solver)
        population.append(individual)

    # Running the genetic algorithm
    for generation in range(generations):
        scores = [evaluate_individual(activation_function, hidden_layer_size, solver) for activation_function, hidden_layer_size, solver in population]
        selected_indices = np.argsort(scores)[::-1][:int(population_size * 0.2)]
        selected_population = [population[i] for i in selected_indices]

        # Creating the new population
        new_population = []
        for _ in range(population_size):
            index1, index2 = np.random.choice(len(selected_population), size=2, replace=False)
            parent1 = selected_population[index1]
            parent2 = selected_population[index2]
            parent1_activation_function, parent1_hidden_layer_size, parent1_solver = parent1
            parent2_activation_function, parent2_hidden_layer_size, parent2_solver = parent2
            child = (parent1_activation_function, parent2_hidden_layer_size, parent2_solver)
            new_population.append(child)

        # Mutating the new population
        population = new_population

    # Finding the best individual
    best_individual = max(population, key=lambda ind: evaluate_individual(ind[0], ind[1], ind[2]))
    return best_individual


### Uitvoeren van het Genetisch Algoritme en Afdrukken van Resultaten

In deze code-cel wordt het genetisch algoritme uitgevoerd om de optimale hyperparameters voor het neurale netwerkmodel te vinden, en worden de resultaten afgedrukt. Hier zijn de belangrijkste elementen die in deze cel gebeuren:

**Uitvoeren van het Genetisch Algoritme**:
- De functie `genetic_algorithm` wordt opgeroepen met twee parameters: `population_size` en `generations`. Hier worden respectievelijk de grootte van de populatie en het aantal generaties voor het genetisch algoritme gespecificeerd.
- Het genetisch algoritme zal meerdere generaties doorlopen en proberen de optimale hyperparameters te vinden op basis van de prestaties van individuen.

**Afdrukken van de Optimale Hyperparameters**:
- Nadat het genetisch algoritme is voltooid, worden de optimale hyperparameters verkregen en opgeslagen in de variabele `best_hyperparameters`.
- Deze optimale hyperparameters worden afgedrukt naar de console met de tekst "Optimal hyperparameters:", gevolgd door de daadwerkelijke hyperparameters.

Deze code is de laatste stap in het proces van hyperparameteroptimalisatie en onthult de optimale configuratie die is gevonden door het genetisch algoritme. Dit is belangrijke informatie om de prestaties van het neurale netwerkmodel te verbeteren.


In [7]:
# Running the genetic algorithm and printing the results
best_hyperparameters = genetic_algorithm(population_size=50, generations=10)
print("Optimal hyperparameters:", best_hyperparameters)


Optimal hyperparameters: ('tanh', 10, 'lbfgs')


### Uitvoeren van het Genetisch Algoritme voor Regressie en de Evaluatie van het Geoptimaliseerde Neurale Netwerk Model Algoritme (met de juiste error metrics)

In deze code-cel wordt het genetisch algoritme voor regressie uitgevoerd om de optimale hyperparameters voor het neurale netwerkmodel te vinden, en worden de resultaten afgedrukt. Hier zijn de belangrijkste elementen die in deze cel gebeuren:

**Uitvoeren van het Genetisch Algoritme**:
- De functie `genetic_algorithm` wordt opgeroepen met twee parameters: `population_size` en `generations`. Hier worden respectievelijk de grootte van de populatie en het aantal generaties voor het genetisch algoritme gespecificeerd.
- Het genetisch algoritme zal meerdere generaties doorlopen en proberen de optimale hyperparameters te vinden op basis van de prestaties van individuen.

**Afdrukken van de Optimale Hyperparameters**:
- Nadat het genetisch algoritme is voltooid, worden de optimale hyperparameters verkregen en opgeslagen in de variabele `best_hyperparameters`.
- Deze optimale hyperparameters worden afgedrukt naar de console met de tekst "Optimale hyperparameters:", gevolgd door de daadwerkelijke hyperparameters.

Deze code is de laatste stap in het proces van hyperparameteroptimalisatie voor regressie en onthult de optimale configuratie die is gevonden door het genetisch algoritme. Dit is belangrijke informatie om de prestaties van het neurale netwerkmodel voor regressie te verbeteren.


In [8]:
# Execute regressiemetrics
best_activation_function, best_hidden_layer_size, best_solver = best_hyperparameters
best_clf = MLPRegressor(hidden_layer_sizes=(best_hidden_layer_size,), activation=best_activation_function, solver=best_solver, max_iter=2000, random_state=42)
best_clf.fit(X_train, y_train)
y_pred = best_clf.predict(X_test)

# Calculate the regressiemetrics based on the predictions
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Make a tabulate list of the regressionmetrics and their values
regression_metrics_values = [
    ["Mean Absolute Error (MAE)", mae],
    ["Mean Squared Error (MSE)", mse],
    ["R-squared (R^2)", r2],
]

# Printing the tabel with regression metrics
print(tabulate(regression_metrics_values, headers=["Regressie Metrieken", "Waarden"], tablefmt="pretty"))


+---------------------------+------------------------+
|    Regressie Metrieken    |        Waarden         |
+---------------------------+------------------------+
| Mean Absolute Error (MAE) |  0.008724607985797736  |
| Mean Squared Error (MSE)  | 0.00016080226541444667 |
|      R-squared (R^2)      |   0.9997699172672925   |
+---------------------------+------------------------+
